In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

import os
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
def camera_calibrate(n_x, n_y, images):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((n_y * n_x, 3), np.float32)
    objp[:, :2] = np.mgrid[0:n_x, 0:n_y].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d points in real world space
    imgpoints = []  # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret1, corners = cv2.findChessboardCorners(gray, (n_x, n_y), None)

        # If found, add object points, image points
        if ret1:
            objpoints.append(objp)
            imgpoints.append(corners)
        else:
            print('Cannot find corners of this chessboard image:', fname)
            print('This set of corners will not be used.')
            
    # calibrate camera
    ret2, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    if ret2:
        print('Camera calibration succeeded.')
        print()
        return [mtx, dist, rvecs, tvecs]
    else:
        print('Camera calibration failed. Please check all of your inputs!')
        print()
        return None

In [8]:
chessboard_images = glob.glob('./camera_cal/calibration*.jpg')
chessboard_aspect_ratio = (9, 6)
cal_params = camera_calibrate(chessboard_aspect_ratio[0], chessboard_aspect_ratio[1], chessboard_images)
cal_params[0]

Cannot find corners of this chessboard image: ./camera_cal\calibration1.jpg
This set of corners will not be used.
Cannot find corners of this chessboard image: ./camera_cal\calibration4.jpg
This set of corners will not be used.
Cannot find corners of this chessboard image: ./camera_cal\calibration5.jpg
This set of corners will not be used.
Camera calibration succeeded.



array([[1.15777930e+03, 0.00000000e+00, 6.67111054e+02],
       [0.00000000e+00, 1.15282291e+03, 3.86128938e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [75]:
def process_image(image, cal_params, label_img_coords, cam_height, offset_values):
    ## 0.Variables configuration
    mtx = cal_params[0]
    dist = cal_params[1]
    offset_x = offset_values[0]
    offset_y = offset_values[1]
    
    ## 1.UNDISTORT each frame
    cal_img = cv2.undistort(image, mtx, dist, None, mtx)
    
    ## 2.RELEVANT Pixel Extraction
    gray = cv2.cvtColor(cal_img,cv2.COLOR_BGR2GRAY)    
    mag_threshold = (60, 255)
    S_threshold = (100, 255)
    
    # magnitude of sobel operator
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    
    mag_sobel = np.sqrt(sobelx**2 + sobely**2)
    abs_sobel = np.absolute(mag_sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    smag_bin_img = np.zeros_like(abs_sobel)
    smag_bin_img[(scaled_sobel >= mag_threshold[0]) & (scaled_sobel <= mag_threshold[1])] = 255
    
    # s-channel of HLS color space
    hls = cv2.cvtColor(cal_img, cv2.COLOR_BGR2HLS)
    S = hls[:, :, 2]
    S_bin_img = np.zeros_like(S)
    S_bin_img[(S >= S_threshold[0]) & (S <= S_threshold[1])] = 255
    
    # stack channels and combine them to one binary image
    stacked_bin_img = np.dstack((smag_bin_img, S_bin_img, S_bin_img))
    combined_bin_img = np.zeros_like(abs_sobel)
    combined_bin_img[(smag_bin_img == 255) | (S_bin_img == 255)] = 255
    
    ## 3.PERSPECTIVE Transformation
    img_size = combined_bin_img.shape[::-1]
    topdown_coords = [[offset_x, offset_y], [img_size[0] - offset_x, offset_y], 
                      [img_size[0] - offset_x, img_size[1] - offset_y], [offset_x, img_size[1] - offset_y]]
    src_pts = np.float32(label_img_coords)
    dst_pts = np.float32(topdown_coords)
    pt_mtx = cv2.getPerspectiveTransform(src_pts, dst_pts)
    
    warped_img = cv2.warpPerspective(combined_bin_img, pt_mtx, img_size, flags=cv2.INTER_NEAREST)
    
    ## 4. SLIDING Windows to find lane pixels, Fitting the polynomial and Calculating the curve radius
    # Hyperparameters for finding lane pixels
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    
    label_world_long_lenth = np.mean([abs(coordTransfrom(label_img_coords[0], cam_height, mtx)[0] - 
                                          coordTransfrom(label_img_coords[3], cam_height, mtx)[0]),
                                      abs(coordTransfrom(label_img_coords[1], cam_height, mtx)[0] - 
                                          coordTransfrom(label_img_coords[2], cam_height, mtx)[0])])
    label_world_lat_length = 3.7  # Just use standard value in US regulations
    meter2pix_fac = (label_world_long_lenth / img_size[1], label_world_lat_length / (img_size[0] - 2*offset_x))
    
    fitted_bin_img, rad_disp_tuple = fit_polynomial(warped_img, hyper_params=(nwindows, margin, minpix), meter2pix_factor=meter2pix_fac)
    
    ## 5. PROJECT the binary images back to calibrated images by inverse perspective transforming
    pt_minv = cv2.getPerspectiveTransform(dst_pts, src_pts)
    
    rectified_img = cv2.warpPerspective(fitted_bin_img, pt_minv, img_size, flags=cv2.INTER_NEAREST)
    rectified_img = cv2.addWeighted(src1=np.uint8(rectified_img), alpha=0.8, src2=cal_img, beta=1, gamma=0)
    
    ## 6. DISPLAY the curve radius and the vehicle deviation in the frame
    rad_text = 'Curvature Radius = {:.2f}m'.format(rad_disp_tuple[0])
    if rad_disp_tuple[1] < 0:
        disp_text = 'Vehicle is {:.2f}m left of center'.format(abs(rad_disp_tuple[1]))
    else:
        disp_text = 'Vehicle is {:.2f}m right of center'.format(abs(rad_disp_tuple[1]))   
    cv2.putText(rectified_img, rad_text, (100, 100), fontFace=5, fontScale=3, color=(255, 255, 255))
    cv2.putText(rectified_img, disp_text, (150, 200), fontFace=5, fontScale=2, color=(255, 255, 255))
    
    return rectified_img

In [74]:
# Two functions almost copied from my exercises
def find_lane_pixels(binary_warped, hyper_params):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = hyper_params[0]
    # Set the width of the windows +/- margin
    margin = hyper_params[1]
    # Set minimum number of pixels found to recenter window
    minpix = hyper_params[2]

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        # Find the four below boundaries of the window 
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((win_y_low <= nonzeroy) &
                          (nonzeroy < win_y_high) &
                          (win_xleft_low <= nonzerox) &
                          (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((win_y_low <= nonzeroy) &
                           (nonzeroy < win_y_high) &
                           (win_xright_low <= nonzerox) &
                           (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped, hyper_params, meter2pix_factor):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped, hyper_params)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # first reset all detected points to black
    out_img[::1] = [0, 0, 0]
    
    # set the freespace as green
    for i_y in ploty:
        out_img[i_y.astype(int), (left_fitx[i_y.astype(int)].astype(int)):(right_fitx[i_y.astype(int)].astype(int))] = [0, 255, 0]
    
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [0, 0, 255]
    out_img[righty, rightx] = [255, 0, 0]    

    # set the color value for pixels on the polynomial instead of plotting them in another figure
    out_img[ploty.astype(int), left_fitx.astype(int)] = [0, 255, 255]
    out_img[ploty.astype(int), right_fitx.astype(int)] = [0, 255, 255]
            
    ## Curvature Radius ##
    ym_per_pix = meter2pix_factor[0] # meters per pixel in y dimension
    xm_per_pix = meter2pix_factor[1] # meters per pixel in x dimension
    
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    y_eval = np.max(ploty*ym_per_pix)
    left_curverad = (1 + (2*left_fit_cr[0]*y_eval + left_fit_cr[1])**2)**1.5 / np.absolute(2*left_fit_cr[0])
    right_curverad = (1 + (2*right_fit_cr[0]*y_eval + right_fit_cr[1])**2)**1.5 / np.absolute(2*right_fit_cr[0])
    
    mean_curverad = (len(leftx)*left_curverad + len(rightx)*right_curverad) / (len(leftx) + len(rightx))
    
    ## Deviation from center ##
    x_disp = 640 - (left_fitx[-1] + right_fitx[-1]) / 2
    
    return out_img, (mean_curverad, xm_per_pix*x_disp)


def coordTransfrom(img_coord, cam_height, mtx):
    '''A simple 3D coordinate transformation matrix.
    '''
    xd = (img_coord[0] - mtx[0,2]) / mtx[0,0]
    yd = (img_coord[1] - mtx[1,2]) / mtx[1,1]
    cam_coord_vec = np.array([[xd], [yd], [1]])
    rtx = np.matrix([[0, -1,  0],
                     [0,  0, -1],
                     [1,  0,  0]])
    
    ret_vec = rtx.T*cam_coord_vec
    
    if ret_vec[2, 0] != 0:
        cz = -cam_height / ret_vec[2, 0]
        return tuple([ret_vec[0, 0] * cz, ret_vec[1, 0] * cz, 0])
    else:
        print('3D-coordinates transformation failed')
        return None

In [77]:
if not os.path.isdir('.\output_videos'):
    os.mkdir('.\output_videos')
video_output_path = '.\output_videos\project_video_output.mp4'
input_video = '.\project_video.mp4'

label_img_coords = [[573, 467], [709, 467], [1007, 667], [293, 667]]
cam_height = 2.05  # A value to be guessed in order to get a reasonable y-estimation to compare with lane width
offset_values = (300, 0)

clip1 = VideoFileClip(input_video)
process_frame = lambda frm: process_image(frm, cal_params, label_img_coords, cam_height, offset_values)
output_clip = clip1.fl_image(process_frame) #NOTE: this function expects color images!!
%time output_clip.write_videofile(video_output_path, audio=False)

t:   0%|                                                                            | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video .\output_videos\project_video_output.mp4.
Moviepy - Writing video .\output_videos\project_video_output.mp4



Moviepy - Done !
Moviepy - video ready .\output_videos\project_video_output.mp4
Wall time: 4min 41s


In [78]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output_path))